<a href="https://colab.research.google.com/github/savancs/product_recommender/blob/tori_branch/BERT_DLAIEF23_PR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports & Setup - Check GPU!

## Packages/Libraries

In [1]:
# Step 1: Install the Hugging Face Transformers library
!pip install transformers
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.2 MB/s eta 0:00:00


In [2]:
# Step 2: Import necessary libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import wandb

from torch.utils.data import DataLoader, TensorDataset

In [3]:
!wandb login --relogin 59add063e6bbf968685833105867f8be02a2191a

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Data

In [4]:
sephora_data = 'https://raw.githubusercontent.com/torrileigh/sephora_cleaned_11.26.23/main/cosmetic_p11.26.23.csv'
sephora_df = pd.read_csv(sephora_data)

sephora_df.head()

,Label,brand,name,price,rank,ingredients,Combination,Dry,Normal,Oily,Sensitive
0,Moisturizer,ALGENIST,GENIUS Ultimate Anti-Aging Cream,$112.00,4.3,"Water, Caprylic/Capric Triglyceride, Hydrogent...",1,1,1,1,1
1,Moisturizer,ALGENIST,Overnight Restorative Cream,$94.00,4.4,"Water, Isopropyl Isostearate, Butyrospermum Pa...",0,0,0,0,0
2,Moisturizer,ALGENIST,SUBLIME DEFENSE Ultra Lightweight UV Defense F...,$28.00,4.4,"Cyclopentasiloxane, Water, Alcohol Denat., Gly...",0,0,0,0,0
3,Moisturizer,ALGENIST,POWER Recharging Night Pressed Serum,$95.00,4.4,"Cocos Nucifera (Coconut) Water, Water (Aqua, E...",1,1,1,1,1
4,Moisturizer,ALGENIST,Firming & Lifting Neck Cream,$98.00,3.9,"Water, Isononyl Isononanoate, Dimethicone, But...",0,0,0,0,0


In [5]:
test_val_data = 'https://raw.githubusercontent.com/savancs/product_recommender/tori_branch/GPT%20Dataset%20PR.csv'
test_val_df = pd.read_csv(test_val_data)

test_val_df.head()

,Input Text,Product Type,Min Price,Max Price,Skin Concerns,Preferred Ingredients,Excluded Ingredients
0,Looking for a sun protect for dry skin $36-$70...,Sun Protect,36.0,70.0,Dry,Peptides,NaN
1,Looking for a eye cream for combination skin $...,Eye Cream,18.0,40.0,Combination,NaN,Phthalates
2,Looking for a moisturizer for dry skin $40-$72...,Moisturizer,40.0,72.0,Dry,NaN,Formaldehyde
3,Looking for a treatment for dry skin $20-$40 w...,Treatment,20.0,40.0,Dry,NaN,Parabens
4,Looking for a face mask for acne-prone skin $3...,Face Mask,37.0,44.0,Acne,Aloe Vera,NaN


In [6]:
annotated_data = 'https://raw.githubusercontent.com/savancs/product_recommender/tori_branch/annotations-2.json'
annotated_df = pd.read_json(annotated_data)

annotated_df.head()

,classes,annotations
0,B-PRODUCT_TYPE,"[Input Text\r, {'entities': []}]"
1,I-PRODUCT_TYPE,[Cleanser with seaweed more than $41 but less ...
2,B-SKIN_CONCERN,[What's a good eye cream to use for skin that'...
3,B-PREF_INGREDIENT,[I need a face mask for acne-prone skin more t...
4,I-PREF_INGREDIENT,[I need a moisturizer that includes Hyaluronic...


In [7]:
print(annotated_df.keys())

Index(['classes', 'annotations'], dtype='object')


In [8]:
# Assuming your data is stored in a variable named `data`
print(annotated_df[:5])  # Print first 5 elements to inspect the structure

             classes                                        annotations
0     B-PRODUCT_TYPE                   [Input Text\r, {'entities': []}]
1     I-PRODUCT_TYPE  [Cleanser with seaweed more than $41 but less ...
2     B-SKIN_CONCERN  [What's a good eye cream to use for skin that'...
3  B-PREF_INGREDIENT  [I need a face mask for acne-prone skin more t...
4  I-PREF_INGREDIENT  [I need a moisturizer that includes Hyaluronic...


# Model Main

Reminder of important names:

sephora_df

test_val_df

annotated_df

## NER - Use this Section!

In [9]:
# Process each annotated text in ner_annotations to align tokens with their labels
# this might involve parsing the structure within each entry in ner_annotations

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def process_annotation(annotation):
    content = annotation[0]
    entities = annotation[1]['entities']
    tokenized_input = tokenizer(content, truncation=True, padding='max_length', max_length=512)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input.input_ids)

    # Initialize labels for each token as 'O'
    aligned_labels = ['O'] * len(tokenized_input['input_ids'])

    # Align labels with BERT tokens
    for start, end, label in entities:
        for i in range(start, end + 1):
            if i < len(tokenized_input['input_ids']):
                prefix = 'B-' if i == start else 'I-'
                aligned_labels[i] = prefix + label

    return tokenized_input['input_ids'], tokenized_input['attention_mask'], aligned_labels

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
def correct_labels(labels):
    corrected_labels = []
    for label_seq in labels:
        corrected_seq = []
        for label in label_seq:
            # Handle the 'O' label correctly
            if label == 'O' or label == 'B-O' or label == 'I-O':
                corrected_label = 'O'
            else:
                parts = label.split('-')
                if len(parts) > 2:
                    # Remove the redundant prefix
                    corrected_label = '-'.join(parts[-2:])
                else:
                    corrected_label = label
            corrected_seq.append(corrected_label)
        corrected_labels.append(corrected_seq)
    return corrected_labels

In [11]:
# Function to convert labels to indices
def convert_labels_to_indices(labels, label_map):
    return [[label_map[label] for label in label_seq] for label_seq in labels]

In [12]:
# Extract classes and annotations
ner_classes = annotated_df['classes'].tolist()
ner_annotations = annotated_df['annotations'].tolist()
print(ner_classes)
print(ner_annotations)

['B-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'B-SKIN_CONCERN', 'B-PREF_INGREDIENT', 'I-PREF_INGREDIENT', 'B-MIN_PRICE', 'B-MAX_PRICE', 'B-EXCL_INGREDIENT', 'I-EXCL_INGREDIENT', 'O']
[['Input Text\r', {'entities': []}], ['Cleanser with seaweed more than $41 but less than $95\r', {'entities': [[0, 8, 'B-PRODUCT_TYPE'], [9, 13, 'O'], [14, 21, 'B-PREF_INGREDIENT'], [22, 26, 'O'], [27, 31, 'O'], [32, 35, 'B-MIN_PRICE'], [36, 39, 'O'], [40, 44, 'O'], [45, 49, 'O'], [50, 53, 'B-MAX_PRICE']]}], ["What's a good eye cream to use for skin that's sensitive?\r", {'entities': [[0, 4, 'O'], [4, 6, 'O'], [7, 8, 'O'], [9, 13, 'O'], [14, 17, 'B-PRODUCT_TYPE'], [18, 23, 'I-PRODUCT_TYPE'], [24, 26, 'O'], [27, 30, 'O'], [31, 34, 'O'], [35, 39, 'O'], [40, 46, 'O'], [47, 56, 'B-SKIN_CONCERN'], [56, 57, 'O']]}], ['I need a face mask for acne-prone skin more than $32 but less than $63.\r', {'entities': [[0, 1, 'O'], [2, 6, 'O'], [7, 8, 'O'], [9, 13, 'B-PRODUCT_TYPE'], [14, 18, 'I-PRODUCT_TYPE'], [19, 22, 'O'], [23, 33

In [13]:
# Process all annotations
processed_data = [process_annotation(annotation) for annotation in ner_annotations]

# Check if processed_data contains None or improperly structured elements
for item in processed_data:
    if item is None or not isinstance(item, tuple) or len(item) != 3:
        print("Found an issue with processed data:", item)


In [14]:
# Extract labels from processed annotations
extracted_labels = [aligned_labels for _, _, aligned_labels in processed_data if aligned_labels is not None]


In [15]:
# Correct the labels
corrected_labels = correct_labels(extracted_labels)

In [16]:
# Create a mapping from string labels to unique integers
label_map = {label: i for i, label in enumerate(ner_classes)}

In [17]:
# Convert the corrected labels to numeric indices
numeric_labels = convert_labels_to_indices(corrected_labels, label_map)

In [18]:
# Convert data to tensors and create a DataLoader
input_ids, attention_mask, _ = zip(*processed_data)
input_ids_tensor = torch.tensor(input_ids, dtype=torch.long)
attention_mask_tensor = torch.tensor(attention_mask, dtype=torch.long)
labels_tensor = torch.tensor(numeric_labels, dtype=torch.long)

dataset = TensorDataset(input_ids_tensor, attention_mask_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=32)

## Tokenizing Input

 - this is more so i understand what the heck is going on, not necessary for training

In [19]:
# Step 3: Load the pre-trained model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# Step 4: Simple test with the model
def test_model(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        logits = model(**inputs).logits
    predictions = torch.nn.functional.softmax(logits, dim=-1)
    return predictions

In [21]:
# Test with a sample text
sample_text = "Hello, this is a test sentence for the BERT model."
predictions = test_model(sample_text, tokenizer, model)
print(predictions)

tensor([[0.6086, 0.3914]])


In [22]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [23]:
# Example input
text = "Looking for a moisturizer for sensitive skin under $30"
original_labels = ["O", "O", "O", "B-PRODUCT_TYPE", "O", "B-SKIN_CONCERN", "I-SKIN_CONCERN", "O", "B-MAX_PRICE"]

# Tokenize the input
tokens = tokenizer.tokenize(text)

# Align the labels with tokens
new_labels = []
label_index = 0
prev_label = None

for token in tokens:
    if token.startswith("##"):
        # For subword tokens, use the previous label, changing B- to I- if necessary
        if prev_label and prev_label.startswith("B-"):
            new_label = "I-" + prev_label.split('-')[-1]
        else:
            new_label = prev_label
    else:
        # This is a new word, assign the label and move to the next label
        if label_index < len(original_labels):
            new_label = original_labels[label_index]
            label_index += 1
        else:
            new_label = "O"

    new_labels.append(new_label)
    prev_label = new_label

# Adjust for special tokens [CLS] and [SEP]
new_labels = ["O"] + new_labels + ["O"]

# Print the tokens with their labels
print(list(zip(["[CLS]"] + tokens + ["[SEP]"], new_labels)))


[('[CLS]', 'O'), ('looking', 'O'), ('for', 'O'), ('a', 'O'), ('moist', 'B-PRODUCT_TYPE'), ('##uri', 'I-PRODUCT_TYPE'), ('##zer', 'I-PRODUCT_TYPE'), ('for', 'O'), ('sensitive', 'B-SKIN_CONCERN'), ('skin', 'I-SKIN_CONCERN'), ('under', 'O'), ('$', 'B-MAX_PRICE'), ('30', 'O'), ('[SEP]', 'O')]


In [24]:
#confirming tokenizes correctly

# Example input
text = "Recommended serum for wrinkle prevention with long-lasting protection"
original_labels = ["O", "B-PRODUCT_TYPE", "I-PRODUCT_TYPE", "O", "B-PURPOSE", "O", "O", "B-LONG_LASTING", "I-LONG_LASTING", "O", "O"]

# Tokenize the input
tokens = tokenizer.tokenize(text)

# Align the labels with tokens
new_labels = []
label_index = 0
prev_label = None

for token in tokens:
    if token.startswith("##"):
        # For subword tokens, use the previous label, changing B- to I- if necessary
        if prev_label and prev_label.startswith("B-"):
            new_label = "I-" + prev_label.split('-')[-1]
        else:
            new_label = prev_label
    else:
        # This is a new word, assign the label and move to the next label
        if label_index < len(original_labels):
            new_label = original_labels[label_index]
            label_index += 1
        else:
            new_label = "O"

    new_labels.append(new_label)
    prev_label = new_label

# Adjust for special tokens [CLS] and [SEP]
new_labels = ["O"] + new_labels + ["O"]

# Print the tokens with their labels
print(list(zip(["[CLS]"] + tokens + ["[SEP]"], new_labels)))

[('[CLS]', 'O'), ('recommended', 'O'), ('serum', 'B-PRODUCT_TYPE'), ('for', 'I-PRODUCT_TYPE'), ('wr', 'O'), ('##ink', 'O'), ('##le', 'O'), ('prevention', 'B-PURPOSE'), ('with', 'O'), ('long', 'O'), ('-', 'B-LONG_LASTING'), ('lasting', 'I-LONG_LASTING'), ('protection', 'O'), ('[SEP]', 'O')]


## Training with BERT

Here's where the fun stuff happens

In [25]:
#define the model:

from transformers import BertForTokenClassification, AdamW

# Define the number of epochs
epochs = 3  # adjust as needed later

# Define the model
model = BertForTokenClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_map),  # Number of distinct labels in your dataset
    output_attentions=False,
    output_hidden_states=False
)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
# set training params

optimizer = AdamW(model.parameters(), lr=1e-5)  # can adjust the learning rate as we train...


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

Set-up WandB before training

In [28]:
# Initialize a new run
wandb.init(project="DLAIE Project F23")

# Track hyperparameters and run metadata
wandb.config = {
  "learning_rate": 1e-5,
  "epochs": 3,
  "batch_size": 32
}

wandb: Currently logged in as: vcrawfordnelson (dlaief23). Use `wandb login --relogin` to force relogin


In [29]:
#training loop
#model is learning from our data
#iterate over the DataLoader, passing data to model
#update model's weights based on the output

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Log batch loss to W&B
        wandb.log({"batch_loss": loss.item()})

        # Backward pass and update
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Calculate the average loss over the epoch
    avg_train_loss = total_loss / len(dataloader)

    # Log average training loss to W&B
    wandb.log({"epoch": epoch, "avg_train_loss": avg_train_loss})

    print(f"Average training loss for epoch {epoch}: {avg_train_loss}")


Average training loss for epoch 0: 2.3062596321105957
Average training loss for epoch 1: 2.1600303649902344
Average training loss for epoch 2: 2.031003475189209


In [30]:
wandb.finish()

avg_train_loss,█▄▁
batch_loss,█▄▁
epoch,▁▅█
avg_train_loss,2.031
batch_loss,2.031
epoch,2


In [ ]:
# save model for future use when we're happy with performance

model.save_pretrained('./model_save')


## Weights & Biases

In [25]:
run = wandb.init(
    # Set the project where this run will be logged
    project="DLAIE Project F23",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": 0.01,
        "epochs": 10,
    })

In [ ]:
#example train, edit later to fine tune

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        # Your training code here: forward pass, loss calculation, backward pass...

        # Log training metrics to W&B
        wandb.log({"train_loss": loss_value, "epoch": epoch})

    # Validation step
    model.eval()
    with torch.no_grad():
        for batch in val_dataloader:
            # Your validation code here...

            # Log validation metrics to W&B
            wandb.log({"val_loss": val_loss_value, "epoch": epoch})


In [26]:
#when training is complete close project

wandb.finish()

## First Attempt Fine Tuning Model for Named Entity Recognition (NER)

Don't use this section it got a little weird and out of order, so redid above

In [14]:
# Process each annotated text in ner_annotations to align tokens with their labels
# this might involve parsing the structure within each entry in ner_annotations

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def process_annotation(annotation):
    content = annotation[0]
    entities = annotation[1]['entities']
    tokenized_input = tokenizer(content, truncation=True, padding='max_length', max_length=512)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input.input_ids)

    # Initialize labels for each token as 'O'
    aligned_labels = ['O'] * len(tokens)

    # Align labels with BERT tokens
    for start, end, label in entities:
        for i in range(start, end + 1):
            if i < len(tokens):
                prefix = 'B-' if i == start else 'I-'
                aligned_labels[i] = prefix + label

    return tokenized_input.input_ids, tokenized_input.attention_mask, aligned_labels

# Process all annotations
processed_data = [process_annotation(annotation) for annotation in ner_annotations]


In [15]:
def correct_labels(labels):
    corrected_labels = []
    for label_seq in labels:
        corrected_seq = []
        for label in label_seq:
            # Handle the 'O' label correctly
            if label == 'O' or label == 'B-O' or label == 'I-O':
                corrected_label = 'O'
            else:
                parts = label.split('-')
                if len(parts) > 2:
                    # Remove the redundant prefix
                    corrected_label = '-'.join(parts[-2:])
                else:
                    corrected_label = label
            corrected_seq.append(corrected_label)
        corrected_labels.append(corrected_seq)
    return corrected_labels

# Correct the labels in your dataset
corrected_labels = correct_labels(labels)

# Continue with converting these corrected labels to indices and tensors
numeric_labels = convert_labels_to_indices(corrected_labels, label_map)
labels_tensor = torch.tensor(numeric_labels, dtype=torch.long)



NameError: ignored

In [13]:
# Extract labels from processed annotations
extracted_labels = [aligned_labels for _, _, aligned_labels in processed_data]

# Correct the labels
corrected_labels = correct_labels(extracted_labels)


In [11]:
# Checking for incorrectly formatted labels
for label_seq in labels:
    for label in label_seq:
        if label not in label_map:
            print("Incorrectly formatted label found:", label)
            # Here, implement the logic to correct the label if necessary


NameError: ignored

In [ ]:
# Create a mapping from string labels to unique integers
label_map = {label: i for i, label in enumerate(ner_classes)}

# Function to convert labels to indices
def convert_labels_to_indices(labels, label_map):
    return [[label_map[label] for label in label_seq] for label_seq in labels]

# Convert the corrected labels to numeric indices
numeric_labels = convert_labels_to_indices(corrected_labels, label_map)

# Convert data to tensors
input_ids, attention_mask, _ = zip(*processed_data)
input_ids_tensor = torch.tensor(input_ids, dtype=torch.long)
attention_mask_tensor = torch.tensor(attention_mask, dtype=torch.long)
labels_tensor = torch.tensor(numeric_labels, dtype=torch.long)

# Create a DataLoader
dataset = TensorDataset(input_ids_tensor, attention_mask_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=32)


whoa this really isn't good!  and not entirely sure why it's adding the extra "B-" and "I-"s in the places it is... regardless, this is happening after it's uploaded it seems.  don't want to mes with changing it in the dataset, so let's write a fundtion to fix... may download and reupload if this ends up slowing things down with 300 datapoints. hopefully it doesn't